In [1]:
# Install all required packages
!pip install openai
!pip install langchain
!pip install chromadb==0.4.3
!pip install tiktoken
!pip install "langchain[docarray]"
!pip install pypdf
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.0/399.0 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
import os
import datetime

import IPython.display
from PIL import Image
import base64
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai
import tkinter as tk

from langchain.llms import OpenAI
from langchain.document_loaders import (
    DataFrameLoader,
    TextLoader,
    PyPDFLoader
)
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter
)
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import (
    DocArrayInMemorySearch,
    Chroma
)
from langchain.chains import (
    RetrievalQA,
    ConversationalRetrievalChain
)
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI


from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


from dotenv import load_dotenv, find_dotenv
from dotenv import load_dotenv, find_dotenv


In [74]:
key = "sk-xxxxxxxxx"
os.environ["OPENAI_API_KEY"] = key

In [4]:
llm = OpenAI()

In [5]:
# load data and preprocess it
def squad_json_to_dataframe(file_path, record_path=['data','paragraphs','qas','answers']):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    """

    file = json.loads(open(file_path).read())
    # parsing different level's in the json file
    js = pd.json_normalize(file, record_path)
    m = pd.json_normalize(file, record_path[:-1])
    r = pd.json_normalize(file,record_path[:-2])

    # combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    m['context'] = idx
    data = m[['id','question','context','answers']].set_index('id').reset_index()
    data['c_id'] = data['context'].factorize()[0]
    return data



In [6]:
# load QA dataset
filename_train = "train-v1.1.json"
filename_test = "dev-v1.1.json"

data_train = squad_json_to_dataframe(filename_train)
data_test = squad_json_to_dataframe(filename_test)



data_train['answers'] = data_train['answers'].apply(lambda x: x[0]['text'] if x else None)
data_test['answers'] = data_test['answers'].apply(lambda x: x[0]['text'] if x else None)

print('Train set:\n')
print(data_train.tail())
print(data_train['context'][0])

print('Test set:\n')
print(data_test.tail())
print(data_test['context'][0])

Train set:

                             id  \
87594  5735d259012e2f140011a09d   
87595  5735d259012e2f140011a09e   
87596  5735d259012e2f140011a09f   
87597  5735d259012e2f140011a0a0   
87598  5735d259012e2f140011a0a1   

                                                question  \
87594  In what US state did Kathmandu first establish...   
87595               What was Yangon previously known as?   
87596  With what Belorussian city does Kathmandu have...   
87597  In what year did Kathmandu create its initial ...   
87598                      What is KMC an initialism of?   

                                                 context  \
87594  Kathmandu Metropolitan City (KMC), in order to...   
87595  Kathmandu Metropolitan City (KMC), in order to...   
87596  Kathmandu Metropolitan City (KMC), in order to...   
87597  Kathmandu Metropolitan City (KMC), in order to...   
87598  Kathmandu Metropolitan City (KMC), in order to...   

                           answers   c_id  
87594      

In [7]:
# create a new data structure combine questions and answers
# add $ at then end so its going to be easier to chunking later
data_train['qa'] = data_train['question'] +data_train['answers']+'$'
data_test['qa'] = data_test['question'] +data_test['answers']+'$'


In [8]:
# load the dataframe into loader
# context
loader_train = DataFrameLoader(data_train, page_content_column="qa")
doc_train = loader_train.load()


loader_test = DataFrameLoader(data_test, page_content_column="qa")
doc_test = loader_test.load()

In [9]:
print(doc_train[0])
print(doc_test[0])



page_content='To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?Saint Bernadette Soubirous$' metadata={'id': '5733be284776f41900661182', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'answers': 'Saint Bernadette Soubirous', 'c_

In [10]:
# get 1/3 of data
doc_train = doc_train[:1000]
doc_test = doc_test[0:1000]

#print(len(doc))

In [11]:
# splitting text into the specific chunck sizes
# defining the overlap size for each chunck


#from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "$",
    chunk_size = 125,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)

#text_splitter = RecursiveCharacterTextSplitter(
#    chunk_size = 50,
#    chunk_overlap = 10
#)

In [12]:
# split our documnet data
splits_train = text_splitter.split_documents(doc_train)
splits_test = text_splitter.split_documents(doc_test)

In [13]:
len(splits_train)

1001

In [14]:
# load the embedding model
embedding = OpenAIEmbeddings(request_timeout=60)

In [15]:
# get the specific gpt model
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)


gpt-3.5-turbo


In [16]:
# create vector database for train and test sets  from data
#db_train = DocArrayInMemorySearch.from_documents(doc_train, embedding)
#db_test = DocArrayInMemorySearch.from_documents(doc_test, embedding)

In [17]:
# print tests
i = 10
for j in range(i+400):
  print('question is: ', data_test['question'][j])
  print('answers is: ', data_test['answers'][j])
  print('########\n\n')


question is:  Which NFL team represented the AFC at Super Bowl 50?
answers is:  Denver Broncos
########


question is:  Which NFL team represented the NFC at Super Bowl 50?
answers is:  Carolina Panthers
########


question is:  Where did Super Bowl 50 take place?
answers is:  Santa Clara, California
########


question is:  Which NFL team won Super Bowl 50?
answers is:  Denver Broncos
########


question is:  What color was used to emphasize the 50th anniversary of the Super Bowl?
answers is:  gold
########


question is:  What was the theme of Super Bowl 50?
answers is:  "golden anniversary"
########


question is:  What day was the game played on?
answers is:  February 7, 2016
########


question is:  What is the AFC short for?
answers is:  American Football Conference
########


question is:  What was the theme of Super Bowl 50?
answers is:  "golden anniversary"
########


question is:  What does AFC stand for?
answers is:  American Football Conference
########


question is:  What

In [18]:
#vectordb_train = db_train

In [19]:
# vector store database
vectordb_train = Chroma.from_documents(
    documents=splits_train,
    embedding=embedding,
)

#vectordb_test = Chroma.from_documents(
#    documents=splits_test,
#    embedding=embedding,
#)

In [20]:
# vector store database
#vectordb_train = Chroma.from_documents(
#    persist_directory='test',
#    documents=splits_train,
#    embedding=embedding,
#)


#vectordb_test.squad_json_to_dataframe(filename_test

In [48]:
# check similarity

question = "What are major topics for this class?"
question = "What color were the Bronco's uniforms in Super Bowl 50?"
docs = vectordb_train.similarity_search(question,k=1)
print(docs[0].metadata['answers'])


Navy Blue and Gold Rush


In [49]:
# create chatbot
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.predict("Hello world!")
llm.predict("What is the Grotto at Notre Dame?")
llm.predict("What team did the Denver Broncos play in  Super Bowl XXXIII?")


'The Denver Broncos played against the Atlanta Falcons in Super Bowl XXXIII.'

In [50]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [51]:
# Build prompt
template = """
You are like an QA agent that you suppose to answer the question that you know.\n
You will always gretting every one at the beging, also you can ask for their name so you will respond back with their name to be more polit.\n
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.\n
Also, if you answered any question except being greedy you can say something like "Do you have any other question that I can help with?".\n
If the person says I don't have any furthur questions, just say something like 'I am always here to help you with question'.
{context}\n

Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
retriever = vectordb_train.as_retriever(search_type="similarity", search_kwargs={"k":8})
#retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .5})
qa_chain = RetrievalQA.from_chain_type(llm,
                                       #retriever=vectordb_train.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .1}),
                                       retriever=retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})




In [52]:
# test topics
question = "Is probability a class topic?"
question = "What color were the Bronco's uniforms in Super Bowl 50?"
result = qa_chain({"query": question})
result["result"]

"I don't know."

In [53]:
# print tests
i = 10
for j in range(i):
  print('question is: ', data_test['question'][j])
  print('answers is: ', data_test['answers'][j])
  print('########\n\n')

question is:  Which NFL team represented the AFC at Super Bowl 50?
answers is:  Denver Broncos
########


question is:  Which NFL team represented the NFC at Super Bowl 50?
answers is:  Carolina Panthers
########


question is:  Where did Super Bowl 50 take place?
answers is:  Santa Clara, California
########


question is:  Which NFL team won Super Bowl 50?
answers is:  Denver Broncos
########


question is:  What color was used to emphasize the 50th anniversary of the Super Bowl?
answers is:  gold
########


question is:  What was the theme of Super Bowl 50?
answers is:  "golden anniversary"
########


question is:  What day was the game played on?
answers is:  February 7, 2016
########


question is:  What is the AFC short for?
answers is:  American Football Conference
########


question is:  What was the theme of Super Bowl 50?
answers is:  "golden anniversary"
########


question is:  What does AFC stand for?
answers is:  American Football Conference
########




In [54]:
# define memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [55]:
#vectordb.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .5})
retriever=vectordb_train.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectordb_train.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .5}),
    memory=memory,
)

In [56]:
question = " How many student news papers are found at Notre Dame?"
question = " What was the theme of Super Bowl 50?"
result = qa({"question": question})

In [57]:
result['answer']

'The theme of Super Bowl 50 was not specified.'

In [58]:

question = "What is the daily student paper at Notre Dame called?"
result = qa({"question": question})

In [59]:
result['answer']

'The daily student paper at Notre Dame is called The Observer.'

In [60]:
# chatbot model initialization
# Build prompt
template = """
You are like an QA agent that you suppose to answer the question that you know.\n
You will always gretting every one at the beging, also you can ask for their name so you will respond back with their name to be more polit.\n
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.\n
Also, if you answered any question except being greedy you can say something like "Do you have any other question that I can help with?".\n
If the person says I don't have any furthur questions, just say something like 'I am always here to help you with question'.
{context}\n

Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)
#vectordb.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .5})
retriever=vectordb_train.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectordb_train.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .2}),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [61]:

question = "What color were the Bronco's uniforms in Super Bowl 50?"
result = qa({"question": question})

In [62]:
print(result['answer'])


The uniforms of the Broncos in Super Bowl 50 were orange. Do you have any other question that I can help with?


In [63]:
i = 0
for t in data_test['question']:
  i += 1
  if i ==10:
    break

  print(t)
print(len(data_test['question']))

#data_test = data_test[:10]

Which NFL team represented the AFC at Super Bowl 50?
Which NFL team represented the NFC at Super Bowl 50?
Where did Super Bowl 50 take place?
Which NFL team won Super Bowl 50?
What color was used to emphasize the 50th anniversary of the Super Bowl?
What was the theme of Super Bowl 50?
What day was the game played on?
What is the AFC short for?
What was the theme of Super Bowl 50?
10570


In [64]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("qa")
evaluator.evaluate_strings(
    prediction="The theme of Super Bowl 50 was 'The Golden Super Bowl.' Do you have any other questions that I can help with?",
    input=" What was the theme of Super Bowl 50?",
    reference="golden anniversary",
)

{'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}

In [65]:
print(len(data_test))


10570


In [66]:
# define evaluation function

container_list = []
#container = {}
for i in range(50):
  container = {}  # Create a new dictionary in each iteration
  #print(data_train['question'][i])
  predict = qa({"question": data_train['question'][i]})
  #print(88888)
  container['prediction'] = predict['answer'].split('Do you have any other question that I can help with?')[0]
  container['input'] = data_train['question'][i]
  container['reference'] = data_train['answers'][i]
  #print(container)
  container_list.append(container)
  #print(container_list)

  #print(f'prediction:' ,container['prediction'])
  #print(f'input:', container['input'])
  #print(f'reference:' ,container['reference'])

  #print('########\n')

#print(container_list)





In [67]:
## check the lenght
#container_list[0]['prediction']
#print(len(container_list))

In [68]:
## print statement to print samples
#for i in range(len(container_list)):

      #prediction=container_list[i]['prediction']
      #input=container_list[i]['input']
      #reference=container_list[i]['reference']

      #print(f'prediction: {prediction}')
      #print(f'input: {input}')
      #print(f'reference: {reference}')

      #print('########\n')


In [69]:
# create the eval list that includes: {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}

eval_list = []
for i in range(len(container_list)):
  eval_results = evaluator.evaluate_strings(
      prediction=container_list[i]['prediction'],
      input=container_list[i]['input'],
      reference=container_list[i]['reference'],
  )
  eval_list.append(eval_results)


In [70]:
print(eval_list)


[{'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}, {'reasoning':

In [71]:
total, correct, idx=0,0,0
for v in eval_list: # loop to compute accuracy

    if v['reasoning'].strip()=='CORRECT':
        correct+=1
    else: # record index for incorrect predictions
        print(f"query: {container_list[idx]['input']}")
        print(f"Answer: {container_list[idx]['reference']}")
        print(f"Prediction: {container_list[idx]['prediction']}")
    total+=1; idx+=1



query: To whom was John B. Kroc married?
Answer: Ray Kroc
Prediction: John B. Kroc was married to Ray Kroc.


In [72]:
print(f'the model prediction accuracy is {correct/total*100} percent')

the model prediction accuracy is 98.0 percent
